In [1]:
# Run this first
!git clone https://github.com/insikk/Grad-CAM-tensorflow.git
!mv Grad-CAM-tensorflow/* .
!wget ftp://mi.eng.cam.ac.uk/pub/mttt2/models/vgg16.npy
!mv vgg16.npy model/

fatal: destination path 'Grad-CAM-tensorflow' already exists and is not an empty directory.


In [0]:
# Run this after cloning
# Replace vanila relu to guided relu to get guided backpropagation.
import tensorflow as tf

from tensorflow.python.framework import ops
from tensorflow.python.ops import gen_nn_ops

@ops.RegisterGradient("GuidedRelu")
def _GuidedReluGrad(op, grad):
    return tf.where(0. < grad, gen_nn_ops._relu_grad(grad, op.outputs[0]), tf.zeros(grad.get_shape()))

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
from model import vgg16
import utils

# Get normalized input. VGG network handles the normalized image internally. 
img1 = utils.load_image("./demo.png")
img2 = utils.load_image("./shihtzu_mypuppy.jpg")

# Load this image and add to visualization
img3 = utils.load_image("./tiger.jpg")


batch1_img = img1.reshape((1, 224, 224, 3))
batch1_label = np.array([1 if i == 242 else 0 for i in range(1000)])  # 1-hot result for Boxer
batch1_label = batch1_label.reshape(1, -1)

batch2_img = img2.reshape((1, 224, 224, 3))
batch2_label = np.array([1 if i == 155 else 0 for i in range(1000)])  # 1-hot result for Shih-Tzu
batch2_label = batch2_label.reshape(1, -1)

#################
# ADD CODE HERE #
#################


##################

#####################
# MAKE CHANGES HERE #
#####################
batch_img = np.concatenate((batch1_img, batch2_img), 0)
batch_label = np.concatenate((batch1_label, batch2_label), 0)

batch_size = 2

#####################

#####################
#   RUN CODE        #
#####################

# for i in range(batch_size):
#     print('See visualization of below category')
#     utils.print_prob(batch_label[i], './synset.txt')

# Create tensorflow graph for evaluation
eval_graph = tf.Graph()

#[batch_size, ]
images = tf.placeholder("float", [batch_size, 224, 224, 3])
labels = tf.placeholder(tf.float32, [batch_size, 1000])
vgg = vgg16.Vgg16()
        
vgg.build(images)
cost = (-1) * tf.reduce_sum(tf.multiply(labels, tf.log(vgg.prob)), axis=1)
print('cost:', cost)
# cost = tf.reduce_sum((vgg.prob - labels) ** 2)
        
        
# gradient for partial linearization. We only care about target visualization class. 
y_c = tf.reduce_sum(tf.multiply(vgg.fc8, labels), axis=1)
print('y_c:', y_c)
# Get last convolutional layer gradient for generating gradCAM visualization
target_conv_layer = vgg.pool5
target_conv_layer_grad = tf.gradients(y_c, target_conv_layer)[0]

# Guided backpropagtion back to input layer
gb_grad = tf.gradients(cost, images)[0]

init = tf.global_variables_initializer()

        
# Run tensorflow 

with tf.Session() as sess:    
    sess.run(init)
    
    prob = sess.run(vgg.prob, feed_dict={images: batch_img})
    
    gb_grad_value, target_conv_layer_value, target_conv_layer_grad_value = sess.run([gb_grad, target_conv_layer, target_conv_layer_grad], feed_dict={images: batch_img, labels: batch_label})
    
    
    for i in range(batch_size):
        utils.print_prob(prob[i], './synset.txt')
        # VGG16 use BGR internally, so we manually change BGR to RGB
        gradBGR = gb_grad_value[i]
        gradRGB = np.dstack((
            gradBGR[:, :, 2],
            gradBGR[:, :, 1],
            gradBGR[:, :, 0],
        ))
        utils.visualize(batch_img[i], target_conv_layer_value[i], target_conv_layer_grad_value[i], gradRGB)
    